# 导入模块

1、机审通过免审核的订单

2、身份证已上传；

3、蚁盾分<30分

4、risk.grade自有评级SABCDEF ;

5、rentOnlineRiskLevelOverall字段，db_credit.risk_alipay_auth_query芝麻租物风控专业版

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 定义数据库连接

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [4]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status, om.user_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租完归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result  
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name, tomt.merchant_id merchant_id_t
,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
,om.buy_service_product,tso.status as service_status 
,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
, tor.update_time, tomt.reason, tp.classify_id, tso.price as 碎屏险价格
, tprm.commodity_premium as 溢价费
,tprm.all_deposit, tprm.all_rental, tprm.total_receivable, tprm.purchase_amount, tprm.contract_price
,topay.payer_user_id, tprm.model_number, tprm.specification, om.relet_status, om.has_actual
,tprm.overdue_type, ym.name zf_name, ym.company_name, om.rent_grade, om.all_sesame_promo_money '总差异化定价优惠金额'
, tprm.liquidated_damages 违约金, om.auto_pay_tag, topc.policy_status, topc.money 保费, tprm.rent_received, topc.policy_url
, cc.scene, tod.source_channel
from  db_digua_business.t_order  om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称 
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 

where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
-- 品牌分类表
left join db_digua_business.t_brand_classify tbc on tbc.id=tod.product_id
-- 商品表
left join db_digua_business.t_product tp on tp.id=tod.product_id
-- 资方表
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
-- 投保订单表
left join db_digua_business.t_order_picc topc on topc.order_id=om.id
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and pa.type!=4
-- and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
-- AND tmer.shop_type !=2  -- 剔除商家
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -7 day )              
-- and om.create_time >= '2024-10-30 00:00:00' 
and  om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -0 day )
-- and  HOUR(om.create_time)< 17
-- and  YEAR(om.create_time) = 2024
-- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-03-01'
-- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-02-28'
;
'''

df_order = query(sql1)
df_order.shape

(380286, 30)

In [5]:
sql2 = '''
SELECT 
om.order_number, tos.money, tos.real_pay_money, tos.sort, tos.refund_date, tos.reality_refund_date, top.create_time as create_time_k
from db_digua_business.t_order  om 
left join db_digua_business.t_order_stages tos on om.id=tos.order_id
left join db_digua_business.t_order_personnel top on om.id=top.order_id
where om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -7 day )              -- 近7天数据
AND om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -0 day )
'''
df_order2 = query(sql2)

df2 = df_order2.copy()
df2.shape

(463111, 7)

In [ ]:
sql_risk = ''' -- risk等级表
select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r from db_credit.risk
'''
df_risk = query(sql_risk)

In [ ]:
sql3 = '''
SELECT id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)
df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)

In [ ]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

# 合并数据

In [6]:
df = df_order.copy()
df.order_id.notnull().sum()

np.int64(380286)

## 日期处理

In [7]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")

df["月份"]=df["下单日期"].dt.month

## 备注信息处理 及关键分类 自段处理

In [8]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')

In [9]:
df = df[df['sku_attributes'].notnull()]
df = df[df["total_describes"].notna()]

In [10]:

df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")


In [11]:
def xinghao_classify(a,b):
    if 'iphone15promax' in a:
        return 'iphone15promax'
    elif 'iphone14promax' in a:
        return 'iphone14promax'
    elif 'iphone15pro' in a:
        return 'iphone15pro'    
    elif 'iphone14pro' in a:
        return 'iphone14pro' 
    elif 'iphone15plus' in a:
        return 'iphone15plus'  
    elif 'iphone15' in a:
        return 'iphone15' 
    elif 'iphone14' in a:
        return 'iphone14'
    elif 'mate60pro' in a:
        return 'mate60pro'

    elif 'iphone14plus' in a:
        return 'iphone14plus'
    elif 'iphone13promax' in a:
        return 'iphone13promax'
    elif 'mate60' in a:
        return 'mate60'
    elif 'watch' in a:
        return 'watch'
    elif 'iphone13pro' in a:
        return 'iphone13pro'
    elif 'vivox100pro' in a:
        return 'vivox100pro'
    elif '小米13pro' in a:
        return '小米13pro'
    elif '小米13' in a:
        return '小米13'
    elif 'iphone12promax' in a:
        return 'iphone12promax'
    elif 'iphone13' in a:
        return 'iphone13'
    elif '小米14pro' in a:
        return '小米14pro'
    elif '小米14' in a:
        return '小米14'
    elif 'iphone12pro' in a:
        return 'iphone12pro'
    elif 'ipad' in a:
        return 'ipad'
    elif 'iphone12' in a:
        return 'iphone12'
    elif '相机' in a:
        return '相机'
    elif '荣耀x50' in a:
        return '荣耀x50'
    elif 'findx6pro' in a:
        return 'findx6pro'
    elif 'xiaomi13' in a:
        return '小米13'
    elif 'vivoiqoo12pro' in a:
        return 'vivoiqoo12pro'
    elif 'oppofindn2' in a:
        return 'oppofindn2'
    elif 'redminote12turbo' in a:
        return 'redminote12turbo'
    elif 'iqoo11' in a:
        return 'iqoo11'
    elif 'vivox100' in a:
        return 'vivox100'
    elif 'mate50pro' in a:
        return 'mate50pro'
    elif 'x90' in a:
        return 'x90'
    elif 'vivoiqooneo8' in a:
        return 'vivoiqooneo8'
    elif '华为pura70pro' in a:
        return '华为pura70pro'
    elif '华为p60' in a:
        return '华为p60'
    elif 'vivoiqoo12' in a:
        return 'vivoiqoo12'
    elif 'x90pro' in a:
        return 'x90pro'
    else :
        return b

df['型号1'] = df.product_name.str.lower().str.replace(' ','')
# df.型号1.value_counts()

df.loc[:,"型号_new"] = df.apply(lambda x:xinghao_classify(x["型号1"],x['product_name']),axis=1)
# df["型号_new"].value_counts()

## 判断进件

In [12]:
df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")

## 判断渠道归属

In [13]:

df.loc[:,"归属渠道"]=df.apply(lambda x:clean.qudao_type(x["来源渠道"],x["activity_name"], x['order_method'], x['channel_type_id']),axis=1)

## 订单预排序去重  剔除内部订单

In [14]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

## 定义审核状态

In [15]:
df = df.merge(df_risk[['id_card_r', 'time_r', 'status_r']], left_on=['id_card_num', '下单日期'], right_on=['id_card_r', 'time_r'], how='left').merge(df_re[['id_card_re', 'time_re', 'status_re']], left_on=['id_card_num', '下单日期'], right_on=['id_card_re', 'time_re'], how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')

# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)


In [16]:

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)

## 区分进件数据源

In [17]:

df["下单时段"]=df["create_time"].astype(str).str[:14]

df_j=df[df["进件"]=="进件"]

### 今日日期

In [18]:
Today= datetime.date.today()

In [19]:
df = clean.status_node(df)

# 去除商家数据

In [20]:
#剔除商家数据只保留自营租机业务数据
df = clean.drop_merchant(df)
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)

In [21]:
df['create_date'] = pd.to_datetime(df['create_date'])
df['create_time'] = pd.to_datetime(df['create_time'])
df['create_hour'] = df['create_time'].dt.hour

In [22]:
# 机审免审的订单占总订单的比例
def pass_rate_ms(df):
    # 进件数量
    df_jj_pass = df[df['是否进件']==1]
    # 机审通过数量
    df_risk_pass = df[(df['是否进件']==1)&(df['机审强拒']!=1)]
    # 免审数量
    df_ms_pass = df_risk_pass[(df_risk_pass.is_vip=='1')&(df_risk_pass.status_result=='0')]
    # 免审比例
    ms_pass_rate = df_ms_pass.shape[0]/df_jj_pass.shape[0]
    ms_pass_rate = np.round(ms_pass_rate,2)

    # 人审比例
    p_pass_rate = (df_risk_pass.shape[0] - df_ms_pass.shape[0])/df_jj_pass.shape[0]
    p_pass_rate =  np.round(p_pass_rate,2)
    # ms_pass_rate = f'{ms_pass_rate:.2}'
    return ms_pass_rate,p_pass_rate



In [23]:
# 有30%左右的客户会进入免人审
ms_pass_rate,p_pass_rate = pass_rate_ms(df)

# 自动扣款

In [24]:
df_js = df[(df['是否进件']==1)&(df['机审强拒']!=1)]
df_js.shape

(1523, 65)

In [25]:
df_mrs = df_js[(df_js.is_vip=='1')&(df_js.status_result=='0')]
df_mrs_count = df_mrs['order_number'].count()
# 自动扣款标记的订单
df_mrs_k = df_mrs[df_mrs['auto_pay_tag']==1]
df_mrs_k_count = df_mrs_k['order_number'].count()
df_mrs_q = df_mrs_k['客户取消'].sum()
df_mrs_r = df_mrs_k['人审拒绝'].sum()
df_mrs_c = df_mrs_k['出库前风控强拒'].sum()
df_mrs_ck = df_mrs_k['是否出库'].sum()
df_mrs_k.shape
df_mrs.shape

(1069, 65)

In [26]:
df_mrs_ks = df_mrs_k[df_mrs_k['是否出库']==1]
# df_mrs_ks = df_mrs_ks[~df_mrs_ks['cancel_reason'].isna()]
# df_mrs_ks[['order_number', '是否出库', 'cancel_reason']]
df_mrs_ks.shape

(89, 65)

# 区分自动扣款和自动扣款成功订单

In [27]:
df3 = pd.merge(df_mrs_k, df2, on='order_number', how='left')
# 获取下单时实付金额
df_order_sum = df3.groupby('order_number').agg({'real_pay_money': 'sum'})
df_order_sum.rename(columns={'real_pay_money': 'pay_money'}, inplace=True)
df4 = pd.merge(df3, df_order_sum, on='order_number', how='left')
# 区分自动扣款成功订单和失败订单
df4_success = df4[df4['pay_money']>1]
df4_fail = df4[df4['pay_money']<=1]

# 获取第一期的实付日期
df4_success_1 = df4_success[df4_success['sort']==1]
df4_success_1.rename(columns={'reality_refund_date': 'sort1_date'}, inplace=True)
# 获取第十二期的实付日期
df4_success_12 = df4_success[df4_success['sort']==12]
df4_success_12.rename(columns={'reality_refund_date': 'sort12_date'}, inplace=True)
df4_success_112 = pd.merge(df4_success_1, df4_success_12, on='order_number', how='left').drop_duplicates()
# 获取第十二期实付日期为空和非空的订单，分别取到最大第十二期和第一期，第一期和下单日期的相差时间
df4_success_112_na = df4_success_112[df4_success_112['sort12_date'].isna()]
df4_success_112_fna = df4_success_112[~df4_success_112['sort12_date'].isna()]

df4_success_112_count1 = df4_success_112_na[(df4_success_112_na['sort1_date']-df4_success_112_na['create_time_x'])<=pd.Timedelta(minutes=5)]
df4_success_112_count2 = df4_success_112_na[(df4_success_112_na['sort1_date']-df4_success_112_na['create_time_x'])>pd.Timedelta(minutes=5)]
df4_success_112_count3 = df4_success_112_fna[(df4_success_112_fna['sort12_date']-df4_success_112_fna['sort1_date'])<=pd.Timedelta(minutes=5)]
df4_success_112_count4 = df4_success_112_fna[(df4_success_112_fna['sort12_date']-df4_success_112_fna['sort1_date'])>pd.Timedelta(minutes=5)]

## 自动扣款成功

In [29]:
# 获取自动扣款成功出库订单
success = pd.concat([df4_success_112_count1, df4_success_112_count3])
df4_success_count = success['order_number'].drop_duplicates().count()
df4_success_count_ck = success[success['是否出库_x']==1]['order_number'].drop_duplicates().count()
df4_success_count_ck

np.int64(79)

## 自动扣款失败

In [30]:
# 获取自动扣款失败订单
fail = pd.concat([df4_success_112_count2, df4_success_112_count4])
fail2 = fail[['order_number', 'sort_x', 'money_x', '是否出库_x']]
fail2.rename(columns={'sort_x': 'sort', 'money_x': 'money', '是否出库_x': '是否出库'}, inplace=True)
df4_fail2 = pd.concat([df4_fail[['order_number', 'sort', 'money', '是否出库']], fail2], axis=0)

In [31]:
df4_fail_count = df4_fail2['order_number'].drop_duplicates().count()
df4_fail_count # 失败订单
df4_fail_1 =  df4_fail2[df4_fail2['sort']==1][df4_fail2[df4_fail2['sort']==1]['money']==1]
df4_fail_1_count = df4_fail_1['order_number'].drop_duplicates().count()
df4_fail_1_count # 首期一元只扣到第一期订单
df4_fail_f1 =  df4_fail2[df4_fail2['sort']==1][df4_fail2[df4_fail2['sort']==1]['money']!=1]
df4_fail_f1_count = df4_fail_f1['order_number'].drop_duplicates().count()
df4_fail_f1_count # 非首期一元只扣到第一期订单
# 自动扣款失败挽回出库订单
df4_fail_count_ck = df4_fail2[df4_fail2['是否出库']==1]['order_number'].drop_duplicates().count()
df4_fail_count_ck

np.int64(10)

In [32]:
# 汇总自动扣款的全部订单
df_auto = pd.DataFrame({
    '自动扣款订单': [df_mrs_k_count],
    '自动扣款成功订单': [df4_success_count],
    '人审拒绝': [df_mrs_r],
    '客户取消': [df_mrs_q],
    '出库前风控强拒': [df_mrs_c],
    '出库': [df_mrs_ck],
    '自动扣款失败订单': [df4_fail_count],
    '自动扣款失败订单（首期一元）': [df4_fail_1_count],
    '自动扣款失败订单（非首期一元）': [df4_fail_f1_count]
})

df_auto['自动扣款比例'] = df_auto['自动扣款订单'] / df_mrs_count
df_auto['自动扣款比例'] = df_auto['自动扣款比例'].apply(lambda x: format(x, '.2%'))

df_auto['成功比例'] = df_auto['自动扣款成功订单'] / df_auto['自动扣款订单']
df_auto['成功比例'] = df_auto['成功比例'].apply(lambda x: format(x, '.2%'))

df_auto['人审拒绝比例'] = df_auto['人审拒绝'] / df_auto['自动扣款订单']
df_auto['人审拒绝比例'] = df_auto['人审拒绝比例'].apply(lambda x: format(x, '.2%'))

df_auto['客户取消比例'] = df_auto['客户取消'] / df_auto['自动扣款订单']
df_auto['客户取消比例'] = df_auto['客户取消比例'].apply(lambda x: format(x, '.2%'))

df_auto['出库前风控强拒比例'] = df_auto['出库前风控强拒'] / df_auto['自动扣款订单']
df_auto['出库前风控强拒比例'] = df_auto['出库前风控强拒比例'].apply(lambda x: format(x, '.2%'))

df_auto['出库比例'] = df_auto['出库'] / df_auto['自动扣款订单']
df_auto['出库比例'] = df_auto['出库比例'].apply(lambda x: format(x, '.2%'))

df_auto['失败比例'] = df_auto['自动扣款失败订单'] / df_auto['自动扣款订单']
df_auto['失败比例'] = df_auto['失败比例'].apply(lambda x: format(x, '.2%'))

df_auto['失败比例（首期一元）'] = df_auto['自动扣款失败订单（首期一元）'] / df_auto['自动扣款订单']
df_auto['失败比例（首期一元）'] = df_auto['失败比例（首期一元）'].apply(lambda x: format(x, '.2%'))

df_auto['失败比例（非首期一元）'] = df_auto['自动扣款失败订单（非首期一元）'] / df_auto['自动扣款订单']
df_auto['失败比例（非首期一元）'] = df_auto['失败比例（非首期一元）'].apply(lambda x: format(x, '.2%'))

df_auto = df_auto[['自动扣款订单', '自动扣款比例', '自动扣款成功订单','成功比例','人审拒绝','人审拒绝比例','客户取消','客户取消比例','出库前风控强拒','出库前风控强拒比例','出库',
                    '出库比例','自动扣款失败订单','失败比例','自动扣款失败订单（首期一元）','失败比例（首期一元）','自动扣款失败订单（非首期一元）','失败比例（非首期一元）']]
df_auto

,自动扣款订单,自动扣款比例,自动扣款成功订单,成功比例,人审拒绝,人审拒绝比例,客户取消,客户取消比例,出库前风控强拒,出库前风控强拒比例,出库,出库比例,自动扣款失败订单,失败比例,自动扣款失败订单（首期一元）,失败比例（首期一元）,自动扣款失败订单（非首期一元）,失败比例（非首期一元）
0,155,14.50%,79,50.97%,21,13.55%,24,15.48%,18,11.61%,89,57.42%,76,49.03%,63,40.65%,13,8.39%


In [33]:
df4_success1 = df4_success[['order_number', '客户取消', '人审拒绝', '出库前风控强拒','待审核', '是否出库']].drop_duplicates()
df4_success_q = df4_success1['客户取消'].sum()
df4_success_r = df4_success1['人审拒绝'].sum()
df4_success_c = df4_success1['出库前风控强拒'].sum()
df4_success_d = df4_success1['待审核'].sum()
df4_success_ck = df4_success1['是否出库'].sum()

# 汇总自动扣款成功的订单
df4_success_group = pd.DataFrame({
    '扣款成功订单': [df4_success_count],
    '自动扣款成功订单': [df4_success_count_ck],
    '自动扣款挽留订单': [df4_fail_count_ck],
    '人审拒绝': [df4_success_r],
    '客户取消': [df4_success_q],
    '出库前风控强拒': [df4_success_c],
    '待审核': [df4_success_d],
    '总出库': [df4_success_count_ck+df4_fail_count_ck]
})

df4_success_group['自动扣款成功订单比例'] = df4_success_group['自动扣款成功订单'] / df4_success_group['扣款成功订单']
df4_success_group['自动扣款成功订单比例'] = df4_success_group['自动扣款成功订单比例'].apply(lambda x: format(x, '.2%'))

df4_success_group['自动扣款挽留订单比例'] = df4_success_group['自动扣款挽留订单'] / df4_fail_count
df4_success_group['自动扣款挽留订单比例'] = df4_success_group['自动扣款挽留订单比例'].apply(lambda x: format(x, '.2%'))

df4_success_group['人审拒绝比例'] = df4_success_group['人审拒绝'] / df4_success_group['扣款成功订单']
df4_success_group['人审拒绝比例'] = df4_success_group['人审拒绝比例'].apply(lambda x: format(x, '.2%'))

df4_success_group['客户取消比例'] = df4_success_group['客户取消'] / df4_success_group['扣款成功订单']
df4_success_group['客户取消比例'] = df4_success_group['客户取消比例'].apply(lambda x: format(x, '.2%'))

df4_success_group['出库前风控强拒比例'] = df4_success_group['出库前风控强拒'] / df4_success_group['扣款成功订单']
df4_success_group['出库前风控强拒比例'] = df4_success_group['出库前风控强拒比例'].apply(lambda x: format(x, '.2%'))

df4_success_group['待审核比例'] = df4_success_group['待审核'] / df4_success_group['扣款成功订单']
df4_success_group['待审核比例'] = df4_success_group['待审核比例'].apply(lambda x: format(x, '.2%'))

df4_success_group['总出库比例'] = df4_success_group['总出库'] / df_mrs_k_count
df4_success_group['总出库比例'] = df4_success_group['总出库比例'].apply(lambda x: format(x, '.2%'))

df4_success_group = df4_success_group[['扣款成功订单', '自动扣款成功订单', '自动扣款成功订单比例', '自动扣款挽留订单', '自动扣款挽留订单比例', '人审拒绝','人审拒绝比例','客户取消','客户取消比例','出库前风控强拒','出库前风控强拒比例', '待审核', '总出库', '总出库比例']]
df4_success_group

,扣款成功订单,自动扣款成功订单,自动扣款成功订单比例,自动扣款挽留订单,自动扣款挽留订单比例,人审拒绝,人审拒绝比例,客户取消,客户取消比例,出库前风控强拒,出库前风控强拒比例,待审核,总出库,总出库比例
0,79,79,100.00%,10,13.16%,0,0.00%,0,0.00%,0,0.00%,0,89,57.42%


# 非自动扣款

In [34]:
df_mrs_fk = df_mrs[df_mrs['auto_pay_tag']!=1]
df_mrs_fk_count = df_mrs_fk['order_number'].count()


## 非免人审

In [35]:
df_fmrs = df_mrs_fk[~df_mrs_fk["decision_result"].str.contains(pat="免人审",regex=False)==True]
df_fmrs['order_number'].count()

np.int64(0)

# 没上传身份证

In [36]:
# 没有办法获取

## iPhone 15Pro_512G

In [37]:
df_mrs_fk['机型'] = df_mrs_fk['product_name'].str.extract(r'(iPhone15Pro)')[0]
df_mrs_fk['机型'].drop_duplicates()
df_mrs_fk_jx_count = df_mrs_fk[(df_mrs_fk['机型']=='iPhone15Pro')&(df_mrs_fk['内存']=='512G')]['order_number'].count()

# 八派|九州

In [38]:
df_mrs_fk_qd_count = df_mrs_fk[(df_mrs_fk['来源渠道']=='八派信息')|((df_mrs_fk['来源渠道']=='九州信息'))]['order_number'].count()

In [39]:
df_fkk = pd.DataFrame({
    '免审订单': [df_mrs_count],
    '非自动扣款': [df_mrs_fk_count],
    'ip15_512': [df_mrs_fk_jx_count],
    '八派九州': [df_mrs_fk_qd_count]
})
df_fkk['非自动扣款比例'] = df_mrs_fk_count/df_mrs_count
df_fkk['非自动扣款比例'] = df_fkk['非自动扣款比例'].apply(lambda x: format(x, '.2%'))

df_fkk['ip15_512比例'] = df_mrs_fk_jx_count/df_mrs_fk_count
df_fkk['ip15_512比例'] = df_fkk['ip15_512比例'].apply(lambda x: format(x, '.2%'))

df_fkk['八派九州比例'] = df_mrs_fk_qd_count/df_mrs_fk_count
df_fkk['八派九州比例'] = df_fkk['八派九州比例'].apply(lambda x: format(x, '.2%'))
df_fkk = df_fkk[['免审订单', '非自动扣款', '非自动扣款比例', 'ip15_512', 'ip15_512比例', '八派九州', '八派九州比例']]

In [40]:
with pd.ExcelWriter(f'F:/需求/七月需求/自动扣款_{Today}.xlsx', engine='openpyxl') as writer:
    df_auto.to_excel(writer, sheet_name='自动扣款汇总', index=False)
    df4_success_group.to_excel(writer, sheet_name='扣款成功转化', index=False)
    df_fkk.to_excel(writer, sheet_name='非自动扣款', index=False)